In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
import env



from huggingface_hub import login
login(env.HF_LOGIN_TOKEN)

# ----------------------------
# Step 1: Connect to Qdrant
# ----------------------------
# Option 1: In-memory (no Docker)
client = QdrantClient(":memory:")

# # Option 2: Local Qdrant server
# client = QdrantClient("http://localhost:6333")

# ----------------------------
# Step 2: Initialize model
# ----------------------------
model = SentenceTransformer("intfloat/multilingual-e5-large")
model.save("models/multilingual-e5-large") #save model locally (do only once)


KeyboardInterrupt: 

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
import torch

client = QdrantClient(host="localhost", port=6333, prefer_grpc=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("models/multilingual-e5-large", device=device)



In [2]:


# from pathlib import Path
# import json
# import pandas as pd

# # ----------------------------
# # Step 3: Load first 100 articles from JSON
# # ----------------------------
# path = Path("../wikiextractor/articles_fr_withLinks.json")
# # ----------------------------
# # Step 3: Load articles from JSON (all lines into DataFrame)
# # ----------------------------
# # ... existing code ...
# ARTICLES_NUMBER = None  # None = load all; set an int to cap when sampling

# def load_jsonl_to_df(path, max_rows=None):
#     rows = []
#     with open(path, "r", encoding="utf-8") as f:
#         for i, line in enumerate(f):
#             if max_rows is not None and i >= max_rows:
#                 break
#             try:
#                 rows.append(json.loads(line))
#             except json.JSONDecodeError:
#                 continue
#     df = pd.DataFrame(rows)
#     # Ensure consistent types and fill missing fields
#     if "id" in df.columns:
#         df["id"] = pd.to_numeric(df["id"], errors="coerce").astype("Int64").astype(int)
#     df["title"] = df.get("title", "").fillna("")
#     df["url"] = df.get("url", "").fillna("")
#     df["text"] = df.get("text", "").fillna("")
#     return df

# df = load_jsonl_to_df(path, max_rows=ARTICLES_NUMBER)
# print(f"✅ Loaded {len(df)} articles into a single DataFrame")
# # ... existing code ...

# # ----------------------------
# # Step 3: Create collection
# # ----------------------------
# vector_size = model.get_sentence_embedding_dimension()

# client.recreate_collection(
#     collection_name="wikipedia_fr",
#     vectors_config=VectorParams(size=vector_size, distance="Cosine"),
# )

# ----------------------------
# Step 3: Load articles from JSON (Polars)
# ----------------------------
from pathlib import Path
import json
import polars as pl

path = Path("../wikiextractor/articles_fr_withLinks.json")

ARTICLES_NUMBER = None  # None = load all; set an int to cap when sampling

def load_jsonl_to_df(path: Path, max_rows: int | None = None) -> pl.DataFrame:
    # If loading the full dataset, use Polars' fast NDJSON reader
    if max_rows is None:
        try:
            df = pl.read_ndjson(str(path))
        except Exception:
            # Fallback if the NDJSON reader hits malformed lines: manual parse
            rows = []
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    try:
                        rows.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
            df = pl.DataFrame(rows)
    else:
        # Limited load: manual parse up to max_rows
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i >= max_rows:
                    break
                try:
                    rows.append(json.loads(line))
                except json.JSONDecodeError:
                    continue
        df = pl.DataFrame(rows)

    # Ensure columns exist and normalize types/nulls
    cols = set(df.columns)

    if "id" in cols:
        # Cast id to Int64
        df = df.with_columns(pl.col("id").cast(pl.Int64))
    else:
        df = df.with_columns(pl.lit(None).cast(pl.Int64).alias("id"))

    if "title" in cols:
        df = df.with_columns(pl.col("title").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("title"))

    if "url" in cols:
        df = df.with_columns(pl.col("url").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("url"))

    if "text" in cols:
        df = df.with_columns(pl.col("text").cast(pl.Utf8).fill_null(""))
    else:
        df = df.with_columns(pl.lit("").alias("text"))

    return df

df = load_jsonl_to_df(path, max_rows=ARTICLES_NUMBER)
print(f"✅ Loaded {len(df)} articles into a single Polars DataFrame")


✅ Loaded 4498441 articles into a single Polars DataFrame


In [3]:
df.write_parquet("articles_fr_withLinks.parquet")

In [2]:
# ... existing code ...
vector_size = model.get_sentence_embedding_dimension()

client.recreate_collection(
    collection_name="wikipedia_fr",
    vectors_config=VectorParams(size=vector_size, distance="Cosine"),
)
# ... existing code ...

/tmp/ipykernel_80404/534072922.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [2]:
import pyarrow.parquet as pq

pf = pq.ParquetFile("articles_fr_withLinks.parquet")
rowgroup_batch = 512
encode_batch = 64


In [3]:

import polars as pl

ARTICLES_NUMBER = 1000


processed = 0
max_rows = ARTICLES_NUMBER
for rg in range(pf.num_row_groups):
    if max_rows is not None and processed >= max_rows:
        break
    table = pf.read_row_group(rg, columns=["id", "title", "url", "text"])
    df = pl.from_arrow(table).with_columns([
        pl.col("id").cast(pl.Int64),
        pl.col("title").fill_null(""),
        pl.col("url").fill_null(""),
        pl.col("text").fill_null("")
    ])
    n = df.height
    remaining = None if max_rows is None else max_rows - processed
    for start in range(0, n, rowgroup_batch):
        length = min(rowgroup_batch, n - start)
        if remaining is not None:
            length = min(length, remaining)
        sub = df.slice(start, length)
        ids = sub["id"].to_list()
        texts = sub["text"].to_list()
        titles = sub["title"].to_list()
        urls = sub["url"].to_list()
        vectors = model.encode(texts, batch_size=encode_batch, convert_to_numpy=True, normalize_embeddings=True)
        points = [
            PointStruct(
                id=int(ids[i]),
                vector=vectors[i].tolist(),
                payload={"id": int(ids[i]), "title": titles[i], "url": urls[i], "text": texts[i]},
            )
            for i in range(len(ids))
        ]
        client.upsert(collection_name="wikipedia_fr", points=points, wait=True)
        processed += length
        if max_rows is not None and processed >= max_rows:
            break

In [4]:
import polars as pl

# ----------------------------
# Step 6: Query similar articles (Polars)
# ----------------------------
query_title = "Math"

qdf = df.filter(pl.col("title").str.to_lowercase() == query_title.lower())
query_text = qdf.select("text").to_series().get(0) if qdf.height > 0 else query_title

query_vector = model.encode([query_text], normalize_embeddings=True)[0]

results = client.search(
    collection_name="wikipedia_fr",
    query_vector=query_vector,
    limit=5
)

print(f"\n🔍 Top matches for '{query_title}':")
for r in results:
    print(f"• {r.payload.get('title')} (score={r.score:.3f})")


🔍 Top matches for 'Math':
• Liste de chimistes (score=0.796)
• Liste alphabétique des sports (score=0.796)
• Bit (score=0.788)
• Espace vectoriel (score=0.786)
• Heidegger (score=0.781)


/tmp/ipykernel_974/569563405.py:13: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [8]:
# ----------------------------
# Step 6: Query similar articles
# ----------------------------
# ... existing code ...
import polars as pl

query_title = "Math"
qdf = df.filter(pl.col("title").str.to_lowercase() == query_title.lower())
query_text = qdf.select("text").to_series().get(0) if qdf.height > 0 else query_title
query_vector = model.encode([query_text], normalize_embeddings=True)[0]
results = client.search(collection_name="wikipedia_fr", query_vector=query_vector, limit=5)
print(f"\n🔍 Top matches for '{query_title}':")
for r in results:
    print(f"• {r.payload.get('title')} (score={r.score:.3f})")


🔍 Top matches for 'Math':
• Algorithme (score=0.779)
• Sigles en physique (score=0.778)
• Abréviations en informatique I (score=0.778)
• Abréviations en informatique A (score=0.778)
• Amenophis IV (score=0.778)


/tmp/ipykernel_80404/809900073.py:11: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name="wikipedia_fr", query_vector=query_vector, limit=5)


In [ ]:
import polars as pl
import numpy as np

def encode_article_text(text):
    words = text.split()
    if not words:
        return np.zeros(model.get_sentence_embedding_dimension(), dtype=np.float32)
    chunks = [" ".join(words[i:i+256]) for i in range(0, len(words), 256)]
    vecs = model.encode([f"passage: {c}" for c in chunks], normalize_embeddings=True)
    return np.mean(vecs, axis=0)

mode = "ids"  # "all" or "ids"
ids_to_process = [7, 9]
k = 5

ids_all = df.select(pl.col("id").cast(pl.Int64)).to_series().to_list()
ids = ids_all if mode == "all" else [i for i in ids_to_process if i in set(ids_all)]

for article_id in ids:
    title_row = df.filter(pl.col("id") == int(article_id)).select("title")
    title = title_row.to_series()[0] if title_row.height > 0 else "(unknown)"
    try:
        points = client.retrieve(collection_name="wikipedia_fr", ids=[int(article_id)], with_payload=True, with_vectors=False)
        if points:
            results = client.recommend(collection_name="wikipedia_fr", positive=[int(article_id)], limit=k)
        else:
            raise ValueError("missing")
    except Exception:
        row = df.filter(pl.col("id") == int(article_id))
        if row.height == 0:
            continue
        text = row.select(pl.col("text").fill_null("")).to_series()[0]
        vec = encode_article_text(text)
        results = client.search(collection_name="wikipedia_fr", query_vector=vec, limit=k)
    print(f"\n🔎 Similar articles for: {title} (id={int(article_id)})")
    for r in results:
        print(f"  • {r.payload.get('title')} (id={r.payload.get('id')}, score={r.score:.3f})")

/tmp/ipykernel_80404/3132525935.py:25: DeprecationWarning: `recommend` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.recommend(collection_name="wikipedia_fr", positive=[int(article_id)], limit=k)



🔎 Similar articles for: Antoine Meillet (id=3)
  • Allier (département) (id=33, score=0.891)
  • Alain Madelin (id=141, score=0.882)
  • Antoine de Saint-Exupéry (id=97, score=0.881)
  • Aveyron (département) (id=41, score=0.871)
  • Alpes-Maritimes (id=35, score=0.869)

🔎 Similar articles for: Algèbre linéaire (id=7)
  • Algèbre générale (id=9, score=0.930)
  • Algèbre de Boole (logique) (id=24, score=0.913)
  • Algorithmique (id=10, score=0.908)
  • Ampère (id=126, score=0.899)
  • Algorithme (id=19, score=0.895)

🔎 Similar articles for: Algèbre générale (id=9)
  • Algèbre linéaire (id=7, score=0.930)
  • Algèbre de Boole (logique) (id=24, score=0.901)
  • Algorithmique (id=10, score=0.899)
  • Algorithme (id=19, score=0.893)
  • Astronomie (id=64, score=0.889)

🔎 Similar articles for: Algorithmique (id=10)
  • Algorithme (id=19, score=0.936)
  • Algèbre linéaire (id=7, score=0.908)
  • Algèbre générale (id=9, score=0.899)
  • Algèbre de Boole (logique) (id=24, score=0.892)
  • Ampè

/tmp/ipykernel_80404/3132525935.py:34: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(collection_name="wikipedia_fr", query_vector=vec, limit=k)



🔎 Similar articles for: Acier (id=172)
  • Alliage (id=50, score=0.935)
  • Azote (id=158, score=0.899)
  • Ampère (id=126, score=0.896)
  • Advanced Micro Devices (id=61, score=0.891)
  • Armement médiéval (id=137, score=0.890)

🔎 Similar articles for: Albert King (id=175)
  • Atari ST (id=120, score=0.892)
  • Akira Kurosawa (id=44, score=0.888)
  • Aston Martin (id=157, score=0.886)
  • Alfred Hitchcock (id=149, score=0.885)
  • Acteur pornographique (id=88, score=0.881)

🔎 Similar articles for: Architecture (id=176)
  • Archéologie sous-marine (id=138, score=0.905)
  • Algorithmique (id=10, score=0.900)
  • Art amarnien (id=116, score=0.898)
  • Astronomie (id=64, score=0.897)
  • Algèbre linéaire (id=7, score=0.892)

🔎 Similar articles for: Anarchisme (id=178)
  • Arlette Laguiller (id=139, score=0.901)
  • Alain Madelin (id=141, score=0.899)
  • Politique en Argentine (id=11, score=0.897)
  • Sid Meier's Alpha Centauri (id=148, score=0.896)
  • Alejandro Toledo (id=142, score=0.

KeyboardInterrupt: 

In [14]:
import polars as pl

cols = ["id", "title", "url", "text"]
df.select(cols).write_csv("articles.csv")

In [ ]:
import polars as pl
import pyarrow.parquet as pq
import pyarrow.csv as pacsv

pf = pq.ParquetFile("articles_fr_withLinks.parquet")
cols = ["id", "title", "url", "text"]
ARTICLES_NUMBER = 100000
processed = 0
first = True
output_path = "articles_stream.csv"

for rg in range(pf.num_row_groups):
    if ARTICLES_NUMBER is not None and processed >= ARTICLES_NUMBER:
        break
    table = pf.read_row_group(rg, columns=cols)
    df_rg = pl.from_arrow(table).with_columns([
        pl.col("id").cast(pl.Int64),
        pl.col("title").fill_null(""),
        pl.col("url").fill_null(""),
        pl.col("text").fill_null("")
    ])
    remaining = None if ARTICLES_NUMBER is None else ARTICLES_NUMBER - processed
    if remaining is not None and df_rg.height > remaining:
        df_rg = df_rg.slice(0, remaining)
    mode = "wb" if first else "ab"
    with open(output_path, mode) as f:
        pacsv.write_csv(df_rg.to_arrow(), f, write_options=pacsv.WriteOptions(include_header=first))
    processed += df_rg.height
    first = False

In [ ]:
import polars as pl
import pyarrow.parquet as pq
from qdrant_client.models import PointStruct

def export_points_df(df: pl.DataFrame, client, collection_name: str, model, encode_batch: int = 64, upsert_batch: int = 512, max_rows: int | None = None):
    n = df.height if max_rows is None else min(df.height, max_rows)
    processed = 0
    while processed < n:
        length = min(upsert_batch, n - processed)
        sub = df.slice(processed, length)
        ids = sub["id"].cast(pl.Int64).to_list()
        titles = sub["title"].fill_null("").to_list()
        urls = sub["url"].fill_null("").to_list()
        texts = sub["text"].fill_null("").to_list()
        vectors = model.encode(texts, batch_size=encode_batch, convert_to_numpy=True, normalize_embeddings=True)
        points = [
            PointStruct(
                id=int(ids[i]),
                vector=vectors[i].tolist(),
                payload={"id": int(ids[i]), "title": titles[i], "url": urls[i], "text": texts[i]},
            )
            for i in range(len(ids))
        ]
        client.upsert(collection_name=collection_name, points=points, wait=True)
        processed += length

NameError: name 'get_similar_articles_by_id' is not defined

In [ ]:
import polars as pl
import pyarrow.parquet as pq
from qdrant_client.models import PointStruct

def export_points_parquet(parquet_path: str, client, collection_name: str, model, encode_batch: int = 64, upsert_batch: int = 512, max_rows: int | None = None):
    pf = pq.ParquetFile(parquet_path)
    processed = 0
    for rg in range(pf.num_row_groups):
        if max_rows is not None and processed >= max_rows:
            break
        table = pf.read_row_group(rg, columns=["id", "title", "url", "text"])
        df = pl.from_arrow(table).with_columns([
            pl.col("id").cast(pl.Int64),
            pl.col("title").fill_null(""),
            pl.col("url").fill_null(""),
            pl.col("text").fill_null("")
        ])
        remaining = None if max_rows is None else max_rows - processed
        n = df.height if remaining is None else min(df.height, remaining)
        inner = 0
        while inner < n:
            length = min(upsert_batch, n - inner)
            sub = df.slice(inner, length)
            ids = sub["id"].to_list()
            titles = sub["title"].to_list()
            urls = sub["url"].to_list()
            texts = sub["text"].to_list()
            vectors = model.encode(texts, batch_size=encode_batch, convert_to_numpy=True, normalize_embeddings=True)
            points = [
                PointStruct(
                    id=int(ids[i]),
                    vector=vectors[i].tolist(),
                    payload={"id": int(ids[i]), "title": titles[i], "url": urls[i], "text": texts[i]},
                )
                for i in range(len(ids))
            ]
            client.upsert(collection_name=collection_name, points=points, wait=True)
            processed += length
            inner += length